<a href="https://colab.research.google.com/github/mohitkhatwani721/bidtrack-commons/blob/main/mohit_groq_colab_5july.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 5.5 MB/s eta 0:00:00


In [2]:
pip install requests

In [9]:
import os
import requests
from google.colab import userdata


groq_api_key = userdata.get("GROQ_API_KEY")

# Define the URL for the Groq API endpoint
url = "https://api.groq.com/openai/v1/chat/completions"

# Set the headers for the API request
headers = {
    "Authorization": f"Bearer {groq_api_key}"
}

# Define the body for the API request
body = {
    "model": "llama-3.1-8b-instant",
    "messages": [
        {
        "role": "system",
        "content": "You are a Product manager and you will answer question with reference to Product building for any app"
      },
      {
        "role": "user",
        "content": "how do i build App"
      }
    ]
}

# Send a POST request to the Groq API
response = requests.post(url, headers=headers, json=body)

# Check if the request was successful
if response.status_code == 200:
    print(response.json()['choices'][0]['message']['content'])
else:
    print("Error:", response.json())

Building an app involves several steps, which I'll outline below. Please note that this is a high-level overview, and the process may vary depending on the type of app you want to build and the platform you're targeting (e.g., mobile, web, desktop).

**Step 1: Idea and Concept**

1. Define your app's purpose, target audience, and unique selling proposition (USP).
2. Conduct market research to understand your competition and identify areas for differentiation.
3. Create a rough wireframe or sketch to visualize your app's layout and core features.

**Step 2: Plan and Design**

1. Develop a detailed wireframe and user flow for each screen.
2. Create a user interface (UI) design concept, including visual elements, typography, and color scheme.
3. Develop a user experience (UX) design concept, focusing on user interaction and engagement.
4. Create a project plan, including timelines, milestones, and resource allocation.

**Step 3: Choose a Development Path**

1. Decide on the app's platform

In [4]:
pip install gradio

In [27]:
import requests
import gradio as gr
from google.colab import userdata

# Groq API Configuration
groq_api_key = userdata.get("GROQ_API_KEY")
url = "https://api.groq.com/openai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {groq_api_key}",
    "Content-Type": "application/json"
}

# Assistant role
system_role = "You are a Product Manager and you will answer questions as a Product Manager."
user_db = {"admin@example.com": "password123"}  # Simulated user DB

# Auth logic
def authenticate(email, password, mode):
    if mode == "Login":
        if email in user_db and user_db[email] == password:
            return gr.update(visible=True), gr.update(visible=False), "", "✅ Login successful"
        else:
            return gr.update(visible=False), gr.update(visible=True), "❌ Invalid email or password", ""
    elif mode == "Sign Up":
        if email in user_db:
            return gr.update(visible=False), gr.update(visible=True), "❌ Email already exists", ""
        user_db[email] = password
        return gr.update(visible=True), gr.update(visible=False), "", "✅ Account created"

# Chat logic
def chat(user_input, history):
    messages = [{"role": "system", "content": system_role}]
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    messages.append({"role": "user", "content": user_input})

    body = {
        "model": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "messages": messages
    }

    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 200:
        reply = response.json()['choices'][0]['message']['content']
        history.append((user_input, reply))
        return history, history
    else:
        error = f"❌ Error: {response.status_code} - {response.text}"
        history.append((user_input, error))
        return history, history

# Reset logic
def reset():
    return [], []

# Toggle password visibility
def toggle_password(view):
    return gr.update(type="text" if view else "password")

# Gradio UI
with gr.Blocks(css="""
#chat-area {
    height: 500px;
    overflow-y: auto;
    background: #F9FAFB;
    border: 1px solid #D1D5DB;
    border-radius: 8px;
    padding: 10px;
}
#send-chat, #reset-chat, #auth-btn {
    background-color: #2563EB !important;
    color: white !important;
    border-radius: 6px;
    padding: 10px 16px;
    font-weight: 600;
}
#send-chat:hover, #reset-chat:hover, #auth-btn:hover {
    background-color: #1D4ED8 !important;
}
""") as demo:

    # Title
    gr.Markdown("## 💬 Product Manager Chatbot")

    # Login/Signup section
    with gr.Column(visible=True) as login_section:
        gr.Markdown("### 🔐 Login or Sign Up to Continue")
        email = gr.Textbox(label="Email", placeholder="you@example.com")
        password = gr.Textbox(label="Password", type="password")
        show_password = gr.Checkbox(label="Show password")
        show_password.change(fn=toggle_password, inputs=show_password, outputs=password)
        mode = gr.Radio(["Login", "Sign Up"], value="Login", label="Mode")
        auth_btn = gr.Button("Submit", elem_id="auth-btn")
        auth_error = gr.Textbox(visible=True, interactive=False, show_label=False)
        auth_success = gr.Textbox(visible=True, interactive=False, show_label=False)

    # Chat section
    with gr.Column(visible=False) as chat_section:
        chatbot = gr.Chatbot(label="Chat", elem_id="chat-area")
        with gr.Row():
            user_input = gr.Textbox(show_label=False, placeholder="Ask a product question...", lines=1, scale=8)
            send_btn = gr.Button("Send", elem_id="send-chat", scale=1)
        reset_btn = gr.Button("Reset Chat", elem_id="reset-chat")
        state = gr.State([])

    # Button logic
    auth_btn.click(fn=authenticate, inputs=[email, password, mode], outputs=[chat_section, login_section, auth_error, auth_success])
    send_btn.click(fn=chat, inputs=[user_input, state], outputs=[chatbot, state])
    user_input.submit(fn=chat, inputs=[user_input, state], outputs=[chatbot, state])
    reset_btn.click(fn=reset, outputs=[chatbot, state])

# Launch app
demo.launch()


/tmp/ipython-input-27-482097069.py:100: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat", elem_id="chat-area")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2863afa309e86fa5ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
#for hugging face  u need to set env seperately and save a app.py file

import os
import requests
import gradio as gr

# Retrieve the API key from the environment variable
groq_api_key = os.getenv("GROQ_API_KEY")


# Define the API endpoint and headers
url = "https://api.groq.com/openai/v1/chat/completions"
headers = {"Authorization": f"Bearer {groq_api_key}"}

# Function to interact with Groq API
def chat_with_groq(user_input):
    body = {
        "model": "llama-3.1-8b-instant",
        "messages": [{"role": "user", "content": user_input}]
    }

    response = requests.post(url, headers=headers, json=body)

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error: {response.json()}"

# Create Gradio interface
interface = gr.Interface(
    fn=chat_with_groq,
    inputs=gr.Textbox(lines=2, placeholder="Ask me anything..."),
    outputs=gr.Textbox(),
    title="Chat with Groq AI (Llama 3.1-8B)",
    description="Type your question below and get a response powered by Groq's Llama 3.1-8B model."
)

# Launch Gradio app
if __name__ == "__main__":
    interface.launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9ea687efb67e33ea86.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
